# Component-wise lambdify

This notebook investigates how to speed up {func}`sympy.lambdify <sympy.utilities.lambdify.lambdify>` by splitting up the expression tree of a complicated expression into components, lambdifying those, and then combining them back again.

In [ ]:
import logging

import ampform
import graphviz
import qrules as q
import sympy as sp
from ampform.dynamics.builder import create_relativistic_breit_wigner_with_ff

## Create amplitude model

First, let's create an amplitude model with {mod}`ampform`. We'll use this model as complicated {class}`sympy.Expr <sympy.core.expr.Expr>` in the rest of this notebooks.

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [ ]:
result = q.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)(980)"],
    allowed_interaction_types=["strong", "EM"],
    formalism_type="canonical-helicity",
)
dot = q.io.asdot(result, collapse_graphs=True)
graphviz.Source(dot)

In [ ]:
model_builder = ampform.get_builder(result)
for name in result.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.generate()

In [ ]:
free_symbols = sorted(model.expression.free_symbols, key=lambda s: s.name)
free_symbols

## Component-wise lambdifying

A {class}`~ampform.helicity.HelicityModel` has the benefit that it comes with {attr}`~ampform.helicity.HelicityModel.components` (intensities and amplitudes) that together form its {attr}`~ampform.helicity.HelicityModel.expression`. Let's separate these components into _amplitude_ and _intensity_.

In [ ]:
amplitudes = {
    name: expr
    for name, expr in model.components.items()
    if name.startswith("A")
}
list(amplitudes)

In [ ]:
intensities = {
    name: expr
    for name, expr in model.components.items()
    if name.startswith("I")
}

In [ ]:
assert len(amplitudes) + len(intensities) == len(model.components)

### Structure of helicity model components

Note that each intensity consists of a subset of these amplitudes. This means that _intensities have a larger expression tree than amplitudes_.

In [ ]:
amplitude_to_symbol = {
    expr: sp.Symbol(f"A{i}") for i, expr in enumerate(amplitudes.values(), 1)
}

In [ ]:
intensity_to_symbol = {
    expr: sp.Symbol(f"I{i}") for i, expr in enumerate(intensities.values(), 1)
}

In [ ]:
intensity_expr = model.expression.subs(intensity_to_symbol, simultaneous=True)
intensity_expr

In [ ]:
dot = sp.dotprint(intensity_expr)
graphviz.Source(dot)

In [ ]:
amplitude_expr = model.expression.subs(amplitude_to_symbol, simultaneous=True)
amplitude_expr

In [ ]:
dot = sp.dotprint(amplitude_expr)
graphviz.Source(dot)

### Performance check

Lambdifying the whole {attr}`HelicityModel.expression <ampform.helicity.HelicityModel.expression>` is slowest. The {func}`~sympy.utilities.lambdify.lambdify` function first prints the expression as a {obj}`str` (!) with (in this case) {mod}`numpy` syntax and then uses {func}`eval` to convert that back to actual {mod}`numpy` objects:

In [ ]:
%%time
np_complete_model = sp.lambdify(free_symbols, model.expression.doit(), "numpy")

Printing to {obj}`str` and converting back with {func}`eval` becomes exponentially slow the larger the expression tree. This means that it's more efficient to lambdify sub-trees of the expression tree separately. Lambdifying the four _intensities_ of this model separately, the effect is not noticeable:

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
%%time
for name, expr in intensities.items():
    logging.info(f"Lambdifying {name}")
    sp.lambdify(free_symbols, expr.doit(), "numpy")

...but each of the eight _amplitudes_ separately does result in a significant speed-up:

In [ ]:
%%time
for name, expr in amplitudes.items():
    logging.info(f"Lambdifying {name}")
    sp.lambdify(free_symbols, expr.doit(), "numpy")